In [1]:
import numpy as np
import pickle

from scipy.sparse import rand
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix

import implementations as imp
import proj1_helpers as ph
import glove as gl

from nltk import pos_tag
#nltk.download()
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [2]:
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
 
print("Stem %s: %s" % ("studying", stemmer.stem("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying", pos="v")))


Stem studying: studi
Lemmatise studying: studying
Lemmatise studying: study


In [3]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
 
s = "This is a simple sentence"
tokens = word_tokenize(s) # Generate list of tokens
tokens_pos = pos_tag(tokens) 
 
print(tokens)

['This', 'is', 'a', 'simple', 'sentence']


In [4]:
#gl.main(20)

loading cooccurrence matrix
6496907 nonzero entries
using nmax = 100 , cooc.max() = 207302
initializing embeddings
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [5]:
glove = np.load('embeddings.npy')
y = pickle.load(open('cooc.pkl', 'rb'))
vocab = pickle.load(open('vocab.pkl', 'rb'))

In [6]:
glove.shape

(21161, 20)

In [7]:
thibault = list(zip(y.row, y.col, y.data))

In [8]:
inv_vocab = {v: k for k, v in vocab.items()}

In [9]:
def split_id_tweet (list_tweets) :
    ids = [x.split(',')[0] for x in list_tweets]
    tweets =  [','.join(x.split(',')[1:]) for x in list_tweets]
    return ids , tweets

In [10]:
tweets_train = []
smile = []
print(len(tweets_train))
with open('twitter-datasets/train_neg_full.txt', encoding='UTF-8') as f :
    read = f.readlines()
    smile = [-1] * len(read)
    tweets_train = read
print(len(tweets_train))
with open('twitter-datasets/train_pos_full.txt', encoding='UTF-8') as f :
    read = f.readlines()
    smile += [1] * len(read)
    tweets_train = np.append(read,tweets_train)
    
print(len(tweets_train))

0
1250000
2500000


In [11]:
tweets_train[:10]

array([ '<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15\n',
       "because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>\n",
       '" <user> just put casper in a box ! " looved the battle ! #crakkbitch\n',
       "<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !\n",
       'visiting my brother tmr is the bestest birthday gift eveerrr ! ! !\n',
       '<user> yay ! ! #lifecompleted . tweet / facebook me to let me know please\n',
       '<user> #1dnextalbumtitle : feel for you / rollercoaster of life . song cocept : life , #yolo , becoming famous ? <3 14 #followmeplz ! <3 x15\n',
       "workin hard or hardly workin rt <user> at hardee's with my future coworker <user>\n",
       "<user> i saw . i'll be replying in a bit .\n",
       'this is were i belong\n'],
      dtype='<U402')

In [12]:
def to_three (idx , line ) :
    res_set= []
    res = []
    for word in line.split(' ') :
        change_nb = vocab.get(word)
        if change_nb != None :
            res_set += [change_nb]
    for change_nb in set(res_set) :
        res += [[1 , idx , change_nb]]
    return res

def to_three_no_res (idx , line ) :
    res = []
    for word in line.split(' ') :
        change_nb = vocab.get(word)
        if change_nb != None :
            res += [[1 , idx , change_nb]]
        
    return res
#tweets_to_sparse = [ np.array(elem) for idx , line in enumerate(tweets,0) for elem in to_three(idx , line)]

def columns ( matrix , i) :
    return [x[i] for x in matrix]


def tweet_to_matrix ( tweets ) :
    tweets_to_sparse = [ np.array(elem) for idx , line in enumerate(tweets,0) for elem in to_three_no_res(idx , line)]
    len(tweets)
    len(glove)
    sparse_tweets = coo_matrix((columns(tweets_to_sparse,0) , (columns(tweets_to_sparse,1),columns(tweets_to_sparse,2))) , shape=(len(tweets), len(glove)))
    print(sparse_tweets.shape)
    return sparse_tweets.dot(glove)/len(tweets)

In [13]:
#sparse_tweets = coo_matrix((columns(0) , (columns(1),columns(2))) , shape=(len(tweets), len(glove)))
#tweets_res = sparse_tweets.dot(glove)
tweets_res = tweet_to_matrix(tweets_train)


(2500000, 21161)


In [14]:
count = 0
count2 = 0
for x in tweets_res :
    count2 +=1
    if  np.count_nonzero(x) == 0 :
        count +=1
count , len(tweets_res) , count/len(tweets_res) , count2

(909, 2500000, 0.0003636, 2500000)

In [15]:
w,loss = imp.least_squares(np.array(smile), tweets_res)

In [16]:
w

array([  31514.63958452,  -27901.15107169,    9906.7080602 ,
       -142201.4842069 ,   22315.45455723,  -50940.86762591,
         34180.25383121,    9050.40040402,   -3038.93689487,
        -31158.54848541,   25514.65142868,  -47528.16036134,
        -51290.97483126,  -21610.69361479,  -24134.04787436,
         18068.75198584,  -42610.54816257,  -22213.88553308,
         41445.03505778,  -47413.61386134])

In [17]:
ids_test=[]
tweets_test=[]
with open('test_data.txt') as f :
    ids , tweets_test = split_id_tweet(f.readlines())
    
tweets_res_test = tweet_to_matrix(tweets_test)


(10000, 21161)


In [18]:
tweets_test[:10]

['sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... <url>\n',
 "<user> shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !\n",
 'i cant stay away from bug thats my baby\n',
 "<user> no ma'am ! ! ! lol im perfectly fine and not contagious anymore lmao\n",
 'whenever i fall asleep watching the tv , i always wake up with a headache\n',
 "<user> he needs to get rid of that thing ! it scares me lol but he don't need a car either . he needs drivers ed again .\n",
 'its whatever . in a terrible mood ( (\n',
 "yesss ! rt <user> <user> thanks jordan , i love you and i'm gonna call you later !\n",
 'my friend <user> text me to check up on me last night .\n',
 '<user> #followback please . when will ur #unitytour come to europe and sweden ? ?\n']

In [19]:
count = 0
count2 = 0
for x in tweets_res_test :
    count2 +=1
    if  np.count_nonzero(x) == 0 :
        count +=1
count , len(tweets_res_test) , count/len(tweets_res_test) , count2

(8, 10000, 0.0008, 10000)

In [20]:
res = tweets_res_test.dot(w)
res = np.sign(res)

In [21]:
imp.create_csv_submission(ids,res,'results_vanilla.csv')

In [22]:
count = 0
for x in res :
    if x  == 0 :
        count +=1
count , len(res) , count/len(res)

(8, 10000, 0.0008)